In [9]:
import csv
import json
import pandas as pd
from datetime import datetime

customerName = {}
with open('/Users/patelrudra/Documents/Medkart/wms/Contacts.csv', mode ='r')as file:
    masterFile = csv.reader(file)    
    for vendor in masterFile:
        customerName[vendor[1]] = vendor[0]
        
vendorItem = {}
with open('/Users/patelrudra/Documents/BOOKS/Item.csv', mode ='r')as file1:
    masterFile = csv.reader(file1)    
    for vendor in masterFile:
        vendorItem[vendor[1]] = vendor[0]
        
with open('/Users/patelrudra/Documents/Medkart/wms/sales26_31.csv', mode ='r')as file:

    csvFile = csv.reader(file)
    data_list = []
    null_voucher_dict = []
    data={}
    count=0
    Bill=True
    notItems=[]
    
    for entry in csvFile:
#         print(entry[4])
        if entry[4] == '':
            if entry[1] == 'Grand Total':
                continue;
            data1 = {
                'item_id':vendorItem[entry[1]],
                'rate':entry[7],
                "quantity":"1",
                "item_order":len(data_list)+1
             }
            data_list.append(data1)
        else:
            data['line_items'] = data_list
            null_voucher_dict.append(data)
            data_list=[]
            date_object = datetime.strptime(entry[0],"%d-%b-%y")
            formatted_date = date_object.strftime("%Y-%m-%d")
            replaced_num=entry[4].replace("_","-")
            data = {
                'date': formatted_date,
                'customer_id': customerName[entry[1]],
                'invoice_number': replaced_num,
                'source_of_supply':"24",
                 "adjustment" : entry[16],
                "is_inclusive_tax": False,
                
             }
    for data in null_voucher_dict:
        json_data = json.dumps(data, indent=4)
        with open("sales6_15.json", "a") as outfile:
            outfile.write(json_data + '\n')  


In [10]:
null_voucher_dict.pop(0)
len(null_voucher_dict)

1377

In [8]:
import requests
import json
url = "https://www.zohoapis.in/books/v3/invoices?ignore_auto_number_generation=true&organization_id=60022053195"
headers = {
  'Content-Type': 'application/json',
  'Authorization': 'Zoho-oauthtoken 1000.38fca0b17f625ca209cfc90531bab513.eec54ef4053d5600d0dfa9912d0fcf99'
}
failedData = []
for data in null_voucher_dict:
    payload = json.dumps(data)

    response = requests.request("POST", url, headers=headers, data=payload)
    if response.status_code >= 200 and response.status_code < 300:  # Assuming a successful response has a status code of 200
        data = response.json()
#         invoiceID = data['invoice']['invoice_id']
#         response = requests.request("POST", f"https://www.zohoapis.in/books/v3/invoices/{invoiceID}/status/sent?organization_id=60021182160", headers=headers, data=[])
    else:

        print(f"API request failed with status code {response.status_code}")
        response_content = response.content.decode('utf-8')
        failedData.append(data['invoice_number'])
        print(response_content)

print("__________________________________")
print("Finished")
print("__________________________________")
print("failed records bill number : ")
print(failedData) 

__________________________________
Finished
__________________________________
failed records bill number : 
[]
